In [43]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [44]:
pwd

'/content'

In [45]:
import os 
import pandas as pd
import seaborn as sns
import torch 
from torch.utils.data import Dataset
from torch import nn

In [46]:
base_dir = "./drive/MyDrive/earthquakes/data/benchmark-earthquake-dataset"
indicators_adeli = "Adeli and Panakkat seismicity indicators"
indicators_reyes = "Reyes et al. seismicity indicators"

aggregation = "1_week" # 1_week, 2_weeks or 1_month

In [47]:
def list_files(base_dir, indicators, aggregation):
    file_names = os.listdir(os.path.join(base_dir, indicators_adeli))
    return [os.path.join(base_dir, indicators, f) for f in file_names if aggregation in f]

adeli_files = list_files(base_dir, indicators_adeli, aggregation)
reyes_files = list_files(base_dir, indicators_reyes, aggregation)

In [48]:
adeli_files

['./drive/MyDrive/earthquakes/data/benchmark-earthquake-dataset/Adeli and Panakkat seismicity indicators/japan_AP_1_week.csv',
 './drive/MyDrive/earthquakes/data/benchmark-earthquake-dataset/Adeli and Panakkat seismicity indicators/hindukush_AP_1_week.csv',
 './drive/MyDrive/earthquakes/data/benchmark-earthquake-dataset/Adeli and Panakkat seismicity indicators/sicily_AP_1_week.csv',
 './drive/MyDrive/earthquakes/data/benchmark-earthquake-dataset/Adeli and Panakkat seismicity indicators/california_AP_1_week.csv',
 './drive/MyDrive/earthquakes/data/benchmark-earthquake-dataset/Adeli and Panakkat seismicity indicators/chile_AP_1_week.csv']

In [49]:
reyes_files

['./drive/MyDrive/earthquakes/data/benchmark-earthquake-dataset/Reyes et al. seismicity indicators/japan_AP_1_week.csv',
 './drive/MyDrive/earthquakes/data/benchmark-earthquake-dataset/Reyes et al. seismicity indicators/hindukush_AP_1_week.csv',
 './drive/MyDrive/earthquakes/data/benchmark-earthquake-dataset/Reyes et al. seismicity indicators/sicily_AP_1_week.csv',
 './drive/MyDrive/earthquakes/data/benchmark-earthquake-dataset/Reyes et al. seismicity indicators/california_AP_1_week.csv',
 './drive/MyDrive/earthquakes/data/benchmark-earthquake-dataset/Reyes et al. seismicity indicators/chile_AP_1_week.csv']

In [50]:
import pandas as pd
df = pd.read_csv(adeli_files[0], sep='\t', index_col='time')

In [51]:
df

,b-value,Mean square deviation,Magnitude deficit,Elapsed days,Mean magnitude,Rate of square root of energy released,Mean time between characteristic events,Coefficient of variation from mean time,mag
time,,,,,,,,,
1974-11-03,0.885901,0.016903,-0.405367,2442.0,5.069,3.188734e+08,49.502163,0.505586,5.1
1974-11-10,0.932661,0.016015,-0.292516,2358.0,5.047,3.139430e+08,49.502163,0.505586,0.0
1974-11-17,0.932661,0.016015,-0.292516,2358.0,5.047,3.139430e+08,49.502163,0.505586,5.8
1974-11-24,0.958862,0.017196,-0.240018,2328.0,5.044,3.128035e+08,49.502163,0.505586,4.5
1974-12-01,0.959048,0.015079,-0.228382,2296.0,5.031,3.102384e+08,49.502163,0.505586,0.0
...,...,...,...,...,...,...,...,...,...
2018-12-09,1.384555,0.002216,-0.012638,330.0,4.737,1.016769e+09,23.793192,1.103193,4.9
2018-12-16,1.386237,0.002305,-0.012535,336.0,4.738,1.000387e+09,23.793192,1.103193,4.7
2018-12-23,1.384085,0.003926,0.005710,336.0,4.726,9.740920e+08,24.009133,1.127253,0.0


In [52]:
sicily_thresh = 3.2
japan_thresh = 5
chile_thresh = 4.5
hindukush_thresh = 4.8
california_thresh = 4

In [53]:
df.mag = (df.mag > japan_thresh).astype(int)

In [54]:
df

,b-value,Mean square deviation,Magnitude deficit,Elapsed days,Mean magnitude,Rate of square root of energy released,Mean time between characteristic events,Coefficient of variation from mean time,mag
time,,,,,,,,,
1974-11-03,0.885901,0.016903,-0.405367,2442.0,5.069,3.188734e+08,49.502163,0.505586,1
1974-11-10,0.932661,0.016015,-0.292516,2358.0,5.047,3.139430e+08,49.502163,0.505586,0
1974-11-17,0.932661,0.016015,-0.292516,2358.0,5.047,3.139430e+08,49.502163,0.505586,1
1974-11-24,0.958862,0.017196,-0.240018,2328.0,5.044,3.128035e+08,49.502163,0.505586,0
1974-12-01,0.959048,0.015079,-0.228382,2296.0,5.031,3.102384e+08,49.502163,0.505586,0
...,...,...,...,...,...,...,...,...,...
2018-12-09,1.384555,0.002216,-0.012638,330.0,4.737,1.016769e+09,23.793192,1.103193,0
2018-12-16,1.386237,0.002305,-0.012535,336.0,4.738,1.000387e+09,23.793192,1.103193,0
2018-12-23,1.384085,0.003926,0.005710,336.0,4.726,9.740920e+08,24.009133,1.127253,0


In [55]:
test_start = "2009-05-03"

df_train = df.loc[:test_start].copy()
df_test = df.loc[test_start:].copy()

print("Test set fraction:", len(df_test) / len(df))

Test set fraction: 0.2194275802254987


In [56]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [57]:
for col in list(df.columns):
    scaler = MinMaxScaler()
    data = np.array(df[col])
    df[col] = scaler.fit_transform(data.reshape(-1, 1))

In [58]:
df

,b-value,Mean square deviation,Magnitude deficit,Elapsed days,Mean magnitude,Rate of square root of energy released,Mean time between characteristic events,Coefficient of variation from mean time,mag
time,,,,,,,,,
1974-11-03,0.116565,0.217208,0.116082,1.000000,0.952038,0.000020,0.944349,0.065036,1.0
1974-11-10,0.147814,0.204806,0.228852,0.965588,0.899281,0.000008,0.944349,0.065036,0.0
1974-11-17,0.147814,0.204806,0.228852,0.965588,0.899281,0.000008,0.944349,0.065036,1.0
1974-11-24,0.165324,0.221304,0.281313,0.953298,0.892086,0.000006,0.944349,0.065036,0.0
1974-12-01,0.165448,0.191737,0.292941,0.940188,0.860911,0.000000,0.944349,0.065036,0.0
...,...,...,...,...,...,...,...,...,...
2018-12-09,0.449811,0.012141,0.508530,0.134781,0.155875,0.001607,0.453024,0.508049,0.0
2018-12-16,0.450935,0.013377,0.508633,0.137239,0.158273,0.001570,0.453024,0.508049,0.0
2018-12-23,0.449497,0.036016,0.526865,0.137239,0.129496,0.001510,0.457151,0.525884,0.0


# Neural Network 

In [67]:
class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[target].values).float()
        self.X = torch.tensor(dataframe[features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

In [68]:
target = 'mag'
features = df.columns
sequence_length = 16

In [69]:
i = 27
sequence_length = 4

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)

X, y = train_dataset[i]
print(X)

tensor([[ 1.0131e+00,  6.0076e-03, -3.0100e-02,  8.1200e+02,  4.9350e+00,
          7.0033e+08,  3.0096e+01,  1.2930e+00,  1.0000e+00],
        [ 1.0138e+00,  5.9503e-03, -3.0246e-02,  8.1700e+02,  4.9360e+00,
          6.9708e+08,  3.0263e+01,  1.3118e+00,  1.0000e+00],
        [ 1.0181e+00,  6.0618e-03, -2.9172e-02,  8.2500e+02,  4.9420e+00,
          6.9506e+08,  2.9442e+01,  1.2782e+00,  0.0000e+00],
        [ 1.0159e+00,  5.9299e-03, -2.9954e-02,  8.3300e+02,  4.9390e+00,
          6.8610e+08,  2.9622e+01,  1.2631e+00,  0.0000e+00]])


In [70]:
from torch.utils.data import DataLoader
torch.manual_seed(99)

train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)

X, y = next(iter(train_loader))
print(X.shape)
print(X)

torch.Size([3, 4, 9])
tensor([[[ 9.3266e-01,  1.6015e-02, -2.9252e-01,  2.3580e+03,  5.0470e+00,
           3.1394e+08,  4.9502e+01,  5.0559e-01,  1.0000e+00],
         [ 9.5886e-01,  1.7196e-02, -2.4002e-01,  2.3280e+03,  5.0440e+00,
           3.1280e+08,  4.9502e+01,  5.0559e-01,  0.0000e+00],
         [ 9.5905e-01,  1.5079e-02, -2.2838e-01,  2.2960e+03,  5.0310e+00,
           3.1024e+08,  4.9502e+01,  5.0559e-01,  0.0000e+00],
         [ 9.5905e-01,  1.5079e-02, -2.2838e-01,  2.2960e+03,  5.0310e+00,
           3.1024e+08,  4.9502e+01,  5.0559e-01,  0.0000e+00]],

        [[ 8.2100e-01,  6.0920e-03,  2.2209e-01,  7.3200e+02,  4.8530e+00,
           8.3365e+08,  4.2669e+01,  9.2443e-01,  1.0000e+00],
         [ 8.2626e-01,  5.4844e-03,  2.1770e-01,  7.3200e+02,  4.8640e+00,
           8.5103e+08,  4.2669e+01,  9.2443e-01,  0.0000e+00],
         [ 8.2626e-01,  5.4844e-03,  2.1770e-01,  7.3200e+02,  4.8640e+00,
           8.5103e+08,  4.2669e+01,  9.2443e-01,  1.0000e+00],
         [

In [108]:
torch.manual_seed(101)

batch_size = 8
sequence_length = 30

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    df_test,
    target=target,
    features=features,
    sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([8, 30, 9])
Target shape: torch.Size([8])


In [109]:
class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_sensors, 
                 hidden_units,
                 num_layers):
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()

        _, (hn, _) = self.lstm(x,)
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.
        out = self.sigmoid(out)

        return out

In [113]:
learning_rate = 1e-3
num_hidden_units = 16

model = ShallowRegressionLSTM(num_sensors=len(features), 
                              hidden_units=num_hidden_units,
                              num_layers=2)

loss_function = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), 
                             lr=learning_rate)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                            patience=3,
                                            verbose=True, 
                                            factor=5e-1)

In [114]:
def train_model(data_loader, model, 
                loss_function, optimizer,):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()

    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    return avg_loss

def test_model(data_loader, model, loss_function):

    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    return avg_loss

In [115]:
n_epochs = 30

print("Untrained test\n--------")
test_loss = test_model(test_loader, model, loss_function)
print(f'Loss: {test_loss}')

for ix_epoch in range(n_epochs):
    train_loss = train_model(train_loader, model, loss_function, optimizer=optimizer)
    test_loss = test_model(test_loader, model, loss_function)
    #scheduler.step(test_loss)
    print(f'Epoch: {ix_epoch} Train loss: {train_loss} Test loss: {test_loss}')

Untrained test
--------
Loss: 0.8260465981438756
Epoch: 0 Train loss: 0.8498240034137152 Test loss: 0.765865795314312
Epoch: 1 Train loss: 0.7798871044564036 Test loss: 0.7329967133700848
Epoch: 2 Train loss: 0.7447102766121383 Test loss: 0.7173395212739706
Epoch: 3 Train loss: 0.7258502479675597 Test loss: 0.7091527059674263
Epoch: 4 Train loss: 0.7158034950231029 Test loss: 0.7045142790302634
Epoch: 5 Train loss: 0.7098826275462597 Test loss: 0.7016122946515679
Epoch: 6 Train loss: 0.705687488602326 Test loss: 0.6996340453624725
Epoch: 7 Train loss: 0.702964530845659 Test loss: 0.6982873165979981
Epoch: 8 Train loss: 0.7011002415576867 Test loss: 0.697301316075027
Epoch: 9 Train loss: 0.6996156631317814 Test loss: 0.6965503385290504
Epoch: 10 Train loss: 0.6984856927816847 Test loss: 0.6959694009274244
Epoch: 11 Train loss: 0.6976042788113113 Test loss: 0.6955152591690421
Epoch: 12 Train loss: 0.6969047441946722 Test loss: 0.6951503800228238
Epoch: 13 Train loss: 0.6963401386695626 T

In [116]:
def predict(data_loader, model):

    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)

    return output


train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

ystar_col = "Model forecast"
df_train[ystar_col] = predict(train_eval_loader, model).numpy()
df_test[ystar_col] = predict(test_loader, model).numpy()

df_out = pd.concat((df_train, df_test))[[target, ystar_col]]
df_out[ystar_col] = (df_out[ystar_col] >= 1).astype(int)

print(df_out)

            mag  Model forecast
time                           
1974-11-03    1               0
1974-11-10    0               0
1974-11-17    1               0
1974-11-24    0               0
1974-12-01    0               0
...         ...             ...
2018-12-09    0               0
2018-12-16    0               0
2018-12-23    0               0
2018-12-30    1               0
2019-01-06    1               0

[2307 rows x 2 columns]


In [ ]:
from sklearn.metrics import recall_score
recall_score(df_out['mag'], df_out['Model forecast'])

In [ ]:
np.max(df_out['mag_lead15']) - np.min(df_out['mag_lead15'])